# Installing dependencies

In [1]:
!pip install pymilvus[milvus-lite] sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 20.4 MB/s eta 0:00:00


### Preparing the model

In [2]:
MODEL_NAME="sentence-transformers/distiluse-base-multilingual-cased-v2"

In [3]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(MODEL_NAME, trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

### Downloading the documents

In [4]:
!unzip hl_rag.zip

Archive:  hl_rag.zip
   creating: hl_rag_raw/
  inflating: hl_rag_raw/10-get_mapname-funkciya-dlya-polucheniya-nazvaniya-karty.html.md  
  inflating: hl_rag_raw/100-ucfirst-funkciya-zmenyaet-registror-pervgogo-simvola-stroki-na-zaglavnyy.html.md  
  inflating: hl_rag_raw/103-funkciya-min.html.md  
  inflating: hl_rag_raw/104-funkciya-max.html.md  
  inflating: hl_rag_raw/105-time-funkciya-poluchaet-vremya-v-tri-raznyh-peremennyh-chms.html.md  
  inflating: hl_rag_raw/106-date-funkciya-poluchenya-daty-v-3-raznyh-peremennyh-gmd.html.md  
  inflating: hl_rag_raw/107-ogranichenie-ispolzovaniya-komandfunkciy-i-td-po-raundam.html.md  
  inflating: hl_rag_raw/108-ogranichenie-ispolzovaniya-komandyfunkcii-v-odnom-raunde.html.md  
  inflating: hl_rag_raw/109-log_amx-funkciya-zapisyvaet-soobschenie-v-log-fayl.html.md  
  inflating: hl_rag_raw/11-plugin_init-funkciya-registracii-plagina.html.md  
  inflating: hl_rag_raw/110-is_map_valid-funkciya-proveryaet-est-li-karta-na-servere.html.md  
  infl

### Preparing the Milvus client instance

In [5]:
from pymilvus import MilvusClient
milvus_client = MilvusClient(uri="/content/milvus_valve_1.0.db")
collection_name = "valve"
if milvus_client.has_collection(collection_name=collection_name):
  milvus_client.drop_collection(collection_name=collection_name)
milvus_client.create_collection(
        collection_name=collection_name,
        dimension=embedding_model.get_sentence_embedding_dimension(),
        metric_type="IP",
        auto_id=True,
        consistency_level="Session"
    )

### Preparing the data

In [9]:
import os
hl_rag_dir = "/content/hl_rag_raw"
data_to_insert = []
for index, i in enumerate(os.listdir(hl_rag_dir)):
  if i.endswith('.md'):
    with open(os.path.join(hl_rag_dir, i), 'r', encoding='utf-8') as file:
      content = file.read()
      data_to_insert.append({"id": index, "vector": embedding_model.encode(content).tolist(), "text": content})


### Inserting the documents into the database

In [10]:
milvus_client.insert(collection_name=collection_name, data=data_to_insert)

{'insert_count': 628, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

### Test Query

In [12]:
query_text = "Как сделать оружие с помощью Half-Life Weapon Mod?"
query_embedding = embedding_model.encode(query_text).tolist()

search_results = milvus_client.search(
        collection_name=collection_name,
        data=[query_embedding],
        limit=5,  # Retrieve top 2 results
        output_fields=["text"]
)
print(search_results)
print("Search results:")
for hit in search_results[0]:
    print(f"Text: {hit['entity']['text']}, Distance: {hit['distance']:.4f}")

data: [[{'id': 329, 'distance': 0.49963241815567017, 'entity': {'text': '# Создание собственного оружия для Haf-Life средствами WeaponMod \n\nВ 2012-м году KORD_12.7 начал написание статьи по разработке собственного оружия с использованием Weapon Mod. Статья так и не была завершена. Исправить это недразумение постараемся мы. В данной статье будут расмотрены методы создания нового оружия для Half-Life средствами модуля Weapon Mod, начиная от простого, заканчивая наворотами.  \n\n## Создание простого оружия \n\n### Регистрация оружия \n\nВ качестве самого первого примера реализуем винтовку M16, имеющую только автоматический режим огня и использующую патроны от MP5.\n\nДля начала подключим необходимый модуль:\n\n```c\n#include <amxmodx>\n#include <hl_wpnmod> // Weapon Mod\n```\n\nТеперь необходимо задать основные параметры нашего оружия \n\n```c\n#define WEAPON_NAME "weapon_m16a1ep" // Название оружия\n#define WEAPON_SLOT\t3 // Слот\n#define WEAPON_POSITION\t5 // Положение в слоте\n#defin